In [ ]:
import os
import re
import io
import sys
import glob
import enum
import json
import dask
import nrrd
import base64
import imageio
import requests
import datetime
import numpy as np
import pandas as pd
import dask.diagnostics
import sqlalchemy as db
import sqlalchemy.orm
import sqlalchemy.ext.declarative
from matplotlib import pyplot as plt

%load_ext autoreload
%autoreload 1

sys.path.append('..')
%aimport opencell.imaging
%aimport opencell.imaging.managers
%aimport opencell.imaging.processors
%aimport opencell.file_utils
%aimport opencell.database.operations

from opencell import constants, file_utils
from opencell.cli import database as db_cli
from opencell.database import models
from opencell.database import operations as ops
from opencell.database import utils as db_utils
from opencell.imaging import utils as im_utils
from opencell.imaging import images, managers, processors, viz

In [ ]:
# autogain all TIFFs in a directory

@dask.delayed
def autoscale_im(src_filepath):
    im = im_utils.autoscale(imageio.imread(src_filepath), p=0.03, dtype='uint8')
    dst_filepath = src_filepath.replace('.tif', '_UINT8.png')
    imageio.imsave(dst_filepath, im)

src_filepaths = glob.glob('/Users/keith.cheveralls/image-data/czML0383-P0009-R01/*.tif')
tasks = [autoscale_im(src_filepath) for src_filepath in src_filepaths]
with dask.diagnostics.ProgressBar():
    dask.compute(*tasks)